# Building a Controller

The following documents the development of a new controller.
In this case we are going to implement an arbitrary controllable storage unit. This
may be a battery, an electrically powered car or some sort of reservoir storage.

## Modelling a Battery

In order to simulate a storage system it is possible to use sgens or loads to model the discharging or charging state. The power of a storage can be positive or negative, so the use of either a sgen or a load is (per definition of the elements) not correct. To overcome this issue, a storage element can be created.

As pandapower is not a time dependend simulation tool and there is no time domain parameter in default power flow calculations, the state of charge (SoC) is not updated during any power flow calculation. 
In Order to update the SoC we build our own storage controller and keep track of the SOC.

State of charge (SoC [\%]) is the level of charge of an electric battery relative to its capacity.


## Controller init
First we start by creating a new file *control/storage_control.py*, containing our new class.

Import and inherit from the parent class `Controller` and override methods you would like to use. Next we write the actual code for the methods.

In [9]:
import pandapower as pp
import control
import pandas as pd
import timeseries as ts

# importing a grid from the library
from pandapower.networks import mv_oberrhein


class Storage(control.basic_controller.Controller):
    """
        Example class of a Storage-Controller. Models an abstract energy storage.
    """
    def __init__(self, net, gid, data_source=None, p_profile=None, in_service=True,
                 recycle=False, order=0, level=0, **kwargs):
        super().__init__(net, in_service=in_service, recycle=recycle, order=order, level=level,
                    initial_powerflow = True, **kwargs)
        
        # read generator attributes from net
        self.gid = gid  # index of the controlled storage
        self.bus = net.storage.at[gid, "bus"]
        self.p_mw = net.storage.at[gid, "p_mw"]
        self.q_mvar = net.storage.at[gid, "q_mvar"]
        self.sn_mva = net.storage.at[gid, "sn_mva"]
        self.name = net.storage.at[gid, "name"]
        self.gen_type = net.storage.at[gid, "type"]
        self.in_service = net.storage.at[gid, "in_service"]
        self.applied = False

        #specific attributes
        self.max_e_mwh = net.storage.at[gid, "max_e_mwh"]
        self.soc_percent = net.storage.at[gid, "soc_percent"] = 0
        
        # profile attributes
        self.data_source = data_source
        self.p_profile = p_profile
        self.last_time_step = None
        
    def is_converged(self):
        # check if controller already was applied
        return self.applied

We choose to represent the storage-unit as a storage
element in pandapower. We start with a function calculating the amout of stored energy:

In [10]:
    def get_stored_ernergy(self):
        # calculating the stored energy
        return self.max_e_mwh * self.soc_percent / 100

Also a first step we want our controller to be able to write its P and Q and state of charge values back to the
data structure net.
   

In [11]:
    def write_to_net(self):
        # write p, q and soc_percent to bus within the net
        self.net.storage.at[self.gid, "p_mw"] = self.p_mw
        self.net.storage.at[self.gid, "q_mvar"] = self.q_mvar
        self.net.storage.at[self.gid, "soc_percent"]= self.soc_percent

Also remember that 'is_converged()' returns the boolean value of convergence:

In case the controller is not yet converged, the control step is executed. In the example it simply
adopts a new value according to the previously calculated target and writes back to the net.

In [12]:
    def control_step(self):
        # Call write_to_net and set the applied variable True
        self.write_to_net()
        self.applied = True


In a time-series simulation the battery should read new power values from a profile and keep track
of its state of charge as depicted below.

In [13]:
    def time_step(self, time):
        # keep track of the soc (assuming time is given in 15min values)
        if self.last_time_step is not None:
            # The amount of Energy produce or consumed in the last timestep is added relative to the 
            # maximum of the possible stored energy
            self.soc_percent += (self.p_mw * (time-self.last_time_step) * 15 / 60) / self.max_e_mwh * 100
        self.last_time_step = time

        # read new values from a profile
        if self.data_source:
            if self.p_profile is not None:
                self.p_mw = self.data_source.get_time_step_value(time_step=time,
                                                                profile_name=self.p_profile)
                
        self.applied = False # reset applied variable

We are now ready to create objects of our newly implemented class and simulate with it!

In [14]:

# loading the network with the usecase 'generation'
net = mv_oberrhein()
pp.runpp(net)

# creating storage unit in the grid, which will be controlled by our controller
store_el = pp.create_storage(net, 30, p_mw = .1, q_mvar = 0, max_e_mwh = 0.1, )

# creating a simple time series
framedata = pd.DataFrame([0.1, .05, 0.1, .005, -0.2, 0], columns=['P'])
datasource = ts.DFData(framedata)

# defining an OutputWriter to track certain variables
ow = ts.OutputWriter(net)

ow.log_variable("res_storage", "p_mw")
ow.log_variable("storage", "soc_percent")


# creating an Object of our new build storage controller
ctrl = Storage(net=net, gid=store_el, data_source=datasource,p_profile='P')

# starting time series simulation
ts.run_timeseries(net, time_steps=range(0, 6), output_writer=ow)

# plotting the state of charge
ow.output['storage.soc_percent'].columns = ['Battery']
ax = ow.output['storage.soc_percent'].plot()
ax.set_xlabel('Time in 15min Steps')
ax.set_ylabel('State of Charge in %')
ax.legend()

print(ow.output)

hp.pandapower.control.util.auxiliary - INFO: Creating controller 0 of type <class '__main__.Storage'> 


Progress: |████████------------------------------------------| 16.7% Complete

hp.pandapower.timeseries.run_time_series - ERROR: ControllerNotConverged at time step 0


ControllerNotConverged: 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 16 15:46:38 2019

@author: jwiemer
"""

import control
import pandapower as pp
import timeseries as ts
import pandas as pd



class Storage(control.basic_controller.Controller):
    """
        Example class of a Storage-Controller. Models an abstract energy storage.
    """
    def __init__(self, net, gid, data_source=None, p_profile=None, in_service=True,
                 recycle=False, order=0, level=0, **kwargs):
        super().__init__(net, in_service=in_service, recycle=recycle, order=order, level=level,
                    initial_powerflow = True, **kwargs)
        
        # read generator attributes from net
        self.gid = gid  # index of the controlled storage
        self.bus = net.storage.at[gid, "bus"]
        self.p_mw = net.storage.at[gid, "p_mw"]
        self.q_mvar = net.storage.at[gid, "q_mvar"]
        self.sn_mva = net.storage.at[gid, "sn_mva"]
        self.name = net.storage.at[gid, "name"]
        self.gen_type = net.storage.at[gid, "type"]
        self.in_service = net.storage.at[gid, "in_service"]
        self.applied = False

        #specific attributes
        self.max_e_mwh = net.storage.at[gid, "max_e_mwh"]
        self.soc_percent = net.storage.at[gid, "soc_percent"] = 0
        
        # profile attributes
        self.data_source = data_source
        self.p_profile = p_profile
        self.last_time_step = None
        
    def get_stored_ernergy(self):
    # do some "complex" calculations
        return self.max_e_mwh * self.soc_percent / 100
    
    def write_to_net(self):
        # write p, q to bus within the net
        self.net.storage.at[self.gid, "p_mw"] = self.p_mw
        self.net.storage.at[self.gid, "q_mvar"] = self.q_mvar
        self.net.storage.at[self.gid, "soc_percent"]= self.soc_percent
        
    def is_converged(self):
    # check if controller already was applied
        return self.applied
    
    def control_step(self):
        # write p, q, soc to storage within the net
        self.write_to_net()
        self.applied = True
        
    def time_step(self, time):
        # keep track of the soc (assuming time is given in 15min values)
        if self.last_time_step is not None:
            self.soc_percent += (self.p_mw * (time-self.last_time_step) * 15 / 60) / self.max_e_mwh * 100
        self.last_time_step = time

        # read new values from a profile
        if self.data_source:
            if self.p_profile is not None:
                self.p_mw = self.data_source.get_time_step_value(time_step=time,
                                                                profile_name=self.p_profile)
                
        self.applied = False # reset applied variable
        

# importing a grid from the library
from pandapower.networks import mv_oberrhein

# loading the network with the usecase 'generation'
net = mv_oberrhein()
pp.runpp(net)

# creating storage unit in the grid, which will be controlled by our controller
store_el = pp.create_storage(net, 30, p_mw = .1, q_mvar = 0, max_e_mwh = 0.1, )

# creating a simple time series
framedata = pd.DataFrame([0.1, .05, 0.1, .005, -0.2, 0], columns=['P'])
datasource = ts.DFData(framedata)

# defining an OutputWriter to track certain variables
ow = ts.OutputWriter(net)

ow.log_variable("res_storage", "p_mw")
ow.log_variable("storage", "soc_percent")


# creating an Object of our new build storage controller
ctrl = Storage(net=net, gid=store_el, data_source=datasource,p_profile='P')

# starting time series simulation
ts.run_timeseries(net, time_steps=range(0, 6), output_writer=ow)

# plotting the state of charge
ow.output['storage.soc_percent'].columns = ['Battery']
ax = ow.output['storage.soc_percent'].plot()
ax.set_xlabel('Time in 15min Steps')
ax.set_ylabel('State of Charge in %')
ax.legend()

#print(ow.output)
        